# Feature Engineering


This notebook shows some feature engineering strategies you may use.

## Imports

In [10]:
import pandas as pd

## Binning


### Numerical binning

We will split happiness into L, M, and H bands.

In [11]:
# Load and inspect the data
df1 = pd.read_csv("fe_binning.csv")
df1.head()

,country,happiness
0,Afghanistan,3.982855
1,Albania,4.606651
2,Argentina,6.697131
3,Armenia,4.348320
4,Australia,7.309061


Here are the descriptive stats for the happiness feature.  We can see that the happiness scores are continuous, between around 2.7 and 7.6.

In [12]:
# Examine the min/max range
df1.describe()

,happiness
count,143.000000
mean,5.404037
std,1.116106
min,2.701591
25%,4.614304
50%,5.344383
75%,6.279204
max,7.603434


We can bin the data into Low, Medium and High bands using the following code:

In [13]:
# Allocate happiness to bins
binned = pd.cut(df1["happiness"], bins = [0, 4.5, 6, 10], labels = ["L","M","H"])

# Add the binned values as a new categorical feature
df1["happiness_band"] = binned

The bins parameter defines the boundaries of the bins.  In this case, I have chosen to split the data into bins containing countries with happiness values of 0 to 4.5, 4.5 to 6 and 6 to 10.

The labels parameter allows us to name each bin, to ensure the low and high bands contain the bottom and top quartiles.  Note there is one less label than bin boundary (because we need 4 boundaries to make 3 bins).

We can now see the new feature:

In [14]:
df1.head()

,country,happiness,happiness_band
0,Afghanistan,3.982855,L
1,Albania,4.606651,M
2,Argentina,6.697131,H
3,Armenia,4.348320,L
4,Australia,7.309061,H


And inspect the number of rows that have ended up in each bin:

In [15]:
# Count of rows in each bin
df1["happiness_band"].value_counts()

M    68
H    44
L    31
Name: happiness_band, dtype: int64

### Categorical binning with Python

We will use a mapping table to add the geographic region that each country is in.

Let's load a mapping table, mapping countries to their region:

In [16]:
# Load the mapping table
mapping = pd.read_csv("country_region.csv")

Join the country on the original data to the region using the mapping table.  We use "left" as the how parameter to specify that all rows in the left table (df1) are included and joined an any matching rows in the right table (mapping).  Any rows in df1 that have no match in mapping will have a null value for the region.

In [17]:
# Merge the mapping table onto the main table, using the country as the key
df1 = pd.merge(df1, mapping, left_on='country', right_on='country', how="left")
df1.head()

,country,happiness,happiness_band,region
0,Afghanistan,3.982855,L,South Asia
1,Albania,4.606651,M,Europe & Central Asia
2,Argentina,6.697131,H,Latin America & Caribbean
3,Armenia,4.348320,L,Europe & Central Asia
4,Australia,7.309061,H,East Asia & Pacific


Let's not assume every row in our data has found a matching region.  Let's check!  We can look at info():

In [18]:
# Check for nulls
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 143 entries, 0 to 142
Data columns (total 4 columns):
country           143 non-null object
happiness         143 non-null float64
happiness_band    143 non-null category
region            143 non-null object
dtypes: category(1), float64(1), object(2)
memory usage: 4.7+ KB


Or check to ensure we have no nulls:

In [19]:
# Check for nulls
df1[df1.region.isnull()]

,country,happiness,happiness_band,region


Note that there are other ways you can perform this mapping (e.g. using a dictionary and the Pandas map function).  Your coding approach will vary according to the task in hand!

## Splitting


### Splitting date and time 

We will split a date and time into its constituent parts.

In [20]:
# Load and inspect the data
df2 = pd.read_csv("fe_splitting.csv")
df2.head()

,borough,property_type,timestamp_of_call
0,Kensington And chelsea,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 16:48
1,Camden,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 22:20
2,Southwark,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 09:51
3,Westminster,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 00:28
4,Barking And dagenham,House - single occupancy,01/01/2017 13:33


One problem to be aware of when loading dates is that Pandas does not automatically recognise dates.  We can confirm this by checking the data types of our features:

In [21]:
# Check the data types
df2.dtypes

borough              object
property_type        object
timestamp_of_call    object
dtype: object

We can use the to_datetime() function to convert the object to a datetime type:

In [22]:
# Convert to datetime type
df2["timestamp_of_call"] = pd.to_datetime(df2["timestamp_of_call"])

Checking again, we can see the timestamp_of_call feature is now a date:

In [23]:
# Check the data types again
df2.dtypes

borough                      object
property_type                object
timestamp_of_call    datetime64[ns]
dtype: object

We can now add some new features by extracting the components of the timestamp_of_call:

In [24]:
# Split the date and time into consituent parts
df2["day"] = df2["timestamp_of_call"].dt.day
df2["month"] = df2["timestamp_of_call"].dt.month
df2["year"] = df2["timestamp_of_call"].dt.year
df2["weekday"] = df2["timestamp_of_call"].dt.weekday
df2["hour"] = df2["timestamp_of_call"].dt.hour

And we can see that we now have our new features:

In [25]:
df2.head()

,borough,property_type,timestamp_of_call,day,month,year,weekday,hour
0,Kensington And chelsea,Purpose Built Flats/Maisonettes - 4 to 9 storeys,2017-01-01 16:48:00,1,1,2017,6,16
1,Camden,Purpose Built Flats/Maisonettes - 4 to 9 storeys,2017-01-01 22:20:00,1,1,2017,6,22
2,Southwark,Purpose Built Flats/Maisonettes - 4 to 9 storeys,2017-01-01 09:51:00,1,1,2017,6,9
3,Westminster,Purpose Built Flats/Maisonettes - 4 to 9 storeys,2017-01-01 00:28:00,1,1,2017,6,0
4,Barking And dagenham,House - single occupancy,2017-01-01 13:33:00,1,1,2017,6,13


It's always worth checking that everything has converted, by checking we have no nulls:

In [26]:
# Check for nulls
df2.isnull().mean()

borough              0.0
property_type        0.0
timestamp_of_call    0.0
day                  0.0
month                0.0
year                 0.0
weekday              0.0
hour                 0.0
dtype: float64

### Splitting compound strings 

We will split a compound string (property_type) on a separator character.

In [27]:
# Load and inspect the data
df3 = pd.read_csv("fe_splitting.csv")
df3.head()

,borough,property_type,timestamp_of_call
0,Kensington And chelsea,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 16:48
1,Camden,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 22:20
2,Southwark,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 09:51
3,Westminster,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 00:28
4,Barking And dagenham,House - single occupancy,01/01/2017 13:33


To split the data, we may infer that we should split at the hyphen.  But we need to be aware some rows may not contain a hyphen, or some rows may contain multiple hyphens.  This could cause a problem.  So let's check our data to see the unique values for property_type we need to deal with:

In [28]:
# Find unique values
df3["property_type"].unique()

array(['Purpose Built Flats/Maisonettes - 4 to 9 storeys',
       'House - single occupancy',
       'Converted Flat/Maisonette - Up to 2 storeys',
       'Purpose Built Flats/Maisonettes - Up to 3 storeys',
       'Purpose Built Flats/Maisonettes - 10 or more storeys',
       'Converted Flat/Maisonettes - 3 or more storeys',
       'Self contained Sheltered Housing',
       'Unlicensed House in Multiple Occupation - Up to 2 storeys',
       'House in Multiple Occupation - 3 or more storeys (not known if licensed)',
       'Student Hall of Residence', 'Other Residential Home',
       'Unlicensed House in Multiple Occupation - 3 or more storeys',
       'Nursing/Care Home/Hospice', "Nurses'/Doctors' accommodation",
       'House in Multiple Occupation - Up to 2 storeys (not known if licensed)',
       'Hotel/motel', "Children's Home",
       'Hostel (e.g. for homeless people)', 'Retirement/Old Persons Home',
       'Licensed House in Multiple Occupation - Up to 2 storeys',
       'Bunga

Looking at these individually, we don't have multiple hyphens, but we do have cases with no hyphen, which will create null entries when we split.  We will need to deal with these later if we want to use this new feature for machine learning.

Let's do the split and confirm the result:

In [29]:
# Split property_type into 2 new columns at a hyphen
df3[['property_type_type', 'property_type_size']] = df3["property_type"].str.split("-",expand=True)
df3.head()

,borough,property_type,timestamp_of_call,property_type_type,property_type_size
0,Kensington And chelsea,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 16:48,Purpose Built Flats/Maisonettes,4 to 9 storeys
1,Camden,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 22:20,Purpose Built Flats/Maisonettes,4 to 9 storeys
2,Southwark,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 09:51,Purpose Built Flats/Maisonettes,4 to 9 storeys
3,Westminster,Purpose Built Flats/Maisonettes - 4 to 9 storeys,01/01/2017 00:28,Purpose Built Flats/Maisonettes,4 to 9 storeys
4,Barking And dagenham,House - single occupancy,01/01/2017 13:33,House,single occupancy


A quick check confirms that the property_type_size column contains nulls as there was no hyphen to split on:

In [30]:
df3.isnull().mean()

borough               0.00000
property_type         0.00000
timestamp_of_call     0.00000
property_type_type    0.00000
property_type_size    0.10007
dtype: float64

We would need to make an informed decision about what to do about these nulls as explained in the data cleansing section of the course!

## One-hot encoding


 ### One-hot enconding with Python
We will one-hot encode the region.

In [31]:
# Load and inspect the data
df4 = pd.read_csv("fe_one_hot.csv")
df4.head()

,country,happiness,region
0,Afghanistan,3.982855,South Asia
1,Albania,4.606651,Europe & Central Asia
2,Argentina,6.697131,Latin America & Caribbean
3,Armenia,4.348320,Europe & Central Asia
4,Australia,7.309061,East Asia & Pacific


We use the Pandas get_dummies() function to convert the required column to dummy variables:

In [32]:
# One-hot encode the region
region_one_hot = pd.get_dummies(df4.region)
region_one_hot.head()

,East Asia & Pacific,Europe & Central Asia,Latin America & Caribbean,Middle East & North Africa,North America,South Asia,Sub-Saharan Africa
0,0,0,0,0,0,1,0
1,0,1,0,0,0,0,0
2,0,0,1,0,0,0,0
3,0,1,0,0,0,0,0
4,1,0,0,0,0,0,0


We can join the new columns back onto the data set, dropping the region column that we just encoded:

In [33]:
# Join one-hot-encoded region back to the data
df4 = df4.join(region_one_hot).drop('region', axis=1)
df4.head()

,country,happiness,East Asia & Pacific,Europe & Central Asia,Latin America & Caribbean,Middle East & North Africa,North America,South Asia,Sub-Saharan Africa
0,Afghanistan,3.982855,0,0,0,0,0,1,0
1,Albania,4.606651,0,1,0,0,0,0,0
2,Argentina,6.697131,0,0,1,0,0,0,0
3,Armenia,4.348320,0,1,0,0,0,0,0
4,Australia,7.309061,1,0,0,0,0,0,0


## Calculated Features


### Calculated features
We will calculate gdp per capita.

In [34]:
# Load and inspect the data
df5 = pd.read_csv("fe_calculated.csv")
df5.head()

,country,gdp_usd,population
0,Afghanistan,1.936297e+10,37172386
1,Albania,1.505888e+10,2866376
2,Argentina,5.184750e+11,44494502
3,Armenia,1.243309e+10,2951776
4,Australia,1.432200e+12,24992369


We can easily perform a calculation on existing features to create a new feature:

In [35]:
df5["gdp_per_capita"] = df5["gdp_usd"] / df5["population"]
df5.head()

,country,gdp_usd,population,gdp_per_capita
0,Afghanistan,1.936297e+10,37172386,520.896603
1,Albania,1.505888e+10,2866376,5253.630064
2,Argentina,5.184750e+11,44494502,11652.563276
3,Armenia,1.243309e+10,2951776,4212.070943
4,Australia,1.432200e+12,24992369,57305.491928
